In [ ]:
import numpy as np
from filters.iekf import IEKF
from tools.objects import Gaussian
from tools.plot import plot2dcov
import mrob
from matplotlib import pyplot as plt

In [ ]:
def ciplot(t, mu, minus_sigma, plus_sigma, x_real, color=None):
    """
    Plots a shaded region on a graph between specified lower and upper confidence intervals (L and U).

    :param t: The time series corresponding to the state.
    :param mu: The predicted state of the variable.
    :param minus_sigma: THe lower bound of the confidence interval.
    :param plus_sigma: The upper bound of the confidence interval.
    :param x_real: The real value of the state variable.
    :param color: Color of the fill inside the lower and upper bound curves (optional).
    :return handle: The handle to the plot of the state variable.
    """

    assert minus_sigma.shape[0] == plus_sigma.shape[0]
    assert t.shape[0] == mu.shape[0]

    plt.fill_between(t, minus_sigma, plus_sigma, color=color, alpha=0.5)
    x_pred, = plt.plot(t, mu)
    x_real, = plt.plot(t, x_real)

    return x_pred, x_real

In [ ]:
a = np.eye(3) @ np.ones(3).reshape(-1, 1)
b = np.eye(3) @ np.ones(3).reshape(-1, 1) * 2
c = np.eye(3) @ np.ones(3).reshape(-1, 1) * 3

In [ ]:
a

In [ ]:
np.hstack((a,b, c))

In [ ]:
mean_prior = np.array([1., 0, 0.])
Sigma_prior = 1e-12 * np.eye(3, 3)
M = 1e-8 * np.eye(3, 3)
Q = 1e-12 * np.eye(3, 3)
initial_state = Gaussian(mean_prior, Sigma_prior)
g = np.array([0, 0, -9.81])
b = np.array([0.01, 0, 0])

In [ ]:
iekf = IEKF(initial_state, M, Q, g, b)

In [ ]:
iekf.mu_bar

In [ ]:
mrob.geometry.SO3(iekf.mu_bar).Ln()

In [ ]:
plot2dcov(iekf.mu_bar[:-1],
            iekf.Sigma_bar[:-1, :-1],
            'red', 3,
            legend='{} - IEKF')

In [ ]:
plot2dcov(iekf.mu[:-1],
            iekf.Sigma[:-1, :-1],
            'blue', 3,
            legend='{} - IEKF')

In [ ]:
iekf.mu

In [ ]:
u = np.array([0.1, 0, 0])
dt = 1

In [ ]:
iekf.predict(u, dt)

In [ ]:
g = np.array([0, 0, -9.81])
iekf.update(g)

In [ ]:
mrob.geometry.SO3([1.1, 0, 0]).mul(mrob.geometry.SO3(u*dt))

In [ ]:
plot2dcov(iekf.mu_bar[:-1],
            iekf.Sigma_bar[:-1, :-1],
            'red', 3,
            legend='{} - IEKF')

In [ ]:
mean_prior = np.array([0, 0, 0.])
Sigma_prior = 1e1 * np.eye(3, 3)
M = 1e0 * np.eye(3, 3)
Q = 1e-2 * np.eye(3, 3)
initial_state = Gaussian(mean_prior, Sigma_prior)
g = np.array([0, 0, -9.81])
u_bias = np.array([0, 0.1, 0])
iekf = IEKF(initial_state, M, Q, g, u_bias)
u = np.array([0.1, 0, 0])
dt = 1
N = 200
preds = np.empty((N, 3))
s_preds = np.empty((N, 3, 3))
ups = np.empty((N, 3))
s_ups = np.empty((N, 3, 3))
gt = np.empty((N, 3))
Ks = np.empty((N, 3, 3))
IVs = np.empty((N, 3))

In [ ]:
for i in range(N):
    iekf.predict(u, dt)
    Ks[i], IVs[i] = iekf.update(mrob.geometry.SO3(np.random.normal(0, 0.1, 3)).R()@mrob.geometry.SO3(iekf.mu_bar).R().T@g)
    preds[i, :] = iekf.mu_bar
    s_preds[i] = iekf.Sigma_bar
    ups[i] = iekf.mu
    s_ups[i] = iekf.Sigma
    gt[i] = mrob.geometry.SO3(iekf.mu_bar).Ln() #np.mod(u*(i+1)+np.pi, 2*np.pi) - np.pi

In [ ]:
mrob.geometry.SO3([0.1, 0, 0]).R()

In [ ]:
plt.plot(IVs)

In [ ]:
plt.plot(np.array([Ks[i] @ IVs[i] for i in range(len(Ks))]))

In [ ]:
plt.plot(ups)

In [ ]:
ups

In [ ]:
s_ups

In [ ]:
plt.plot(gt)

In [ ]:
plt.plot(gt - ups)

In [ ]:
def plot_covs(states, covs, state_i, gt):
    sigma = np.sqrt(covs[:, state_i, state_i])
    minus_sigma = states[:, state_i] - sigma
    plus_sigma = states[:, state_i] + sigma
    plt.figure(figsize=(14,8))
    t = np.array(range(len(states)))
    handles = ciplot(t, states[:, state_i], minus_sigma, plus_sigma, gt[:, state_i])


    plt.title('Direct Integration of Controls')
    plt.legend(handles, ['Estimated angle', 'Ground Truth'])
    plt.xlabel('Time')
    plt.ylabel('Translational Velocity')
    plt.show()

In [ ]:
plot_covs(preds, s_preds, 0, gt)
plot_covs(preds, s_preds, 1, gt)
plot_covs(preds, s_preds, 2, gt)

In [ ]:
plot_covs(ups, s_ups, 0, gt)
plot_covs(ups, s_ups, 1, gt)
plot_covs(ups, s_ups, 2, gt)